In [1]:
from experiment import Exp
import yaml
import os
import torch
from ptflops import get_model_complexity_info
import sys

sys.path.append("../../")

In [2]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [3]:
args = dotdict()
# TODO change the path as relative path
args.to_save_path = r"../../data/Run_logs"
args.freq_save_path = r"../../data/Freq_data"
args.window_save_path = r"../../data/Sliding_window"
args.root_path = r"../.."

args.drop_transition = False
args.datanorm_type = "standardization"  # None ,"standardization", "minmax"
args.filter_scaling_factor = 1
args.batch_size = 256
args.shuffle = True
args.drop_last = False
args.train_vali_quote = 0.90
# training setting
args.train_epochs = 150
args.learning_rate = 0.001
args.learning_rate_patience = 7
args.learning_rate_factor = 0.1
args.early_stop_patience = 15

args.use_gpu = True if torch.cuda.is_available() else False
args.gpu = 0
args.use_multi_gpu = False

args.optimizer = "Adam"
args.criterion = "CrossEntropy"
args.seed = 1
args.data_name = 'harvar'
args.wavelet_filtering = False
args.wavelet_filtering_regularization = False
args.wavelet_filtering_finetuning = False
args.wavelet_filtering_finetuning_percent = 0.5
args.wavelet_filtering_learnable = False
args.wavelet_filtering_layernorm = False
args.regulatization_tradeoff = 0
args.number_wavelet_filtering = 12
args.difference = False
args.filtering = False
args.magnitude = False
args.weighted_sampler = True
args.pos_select = None
args.sensor_select = None
args.representation_type = "time"
args.exp_mode = "LOCV"

config_file = open('../../configs/data.yaml', mode='r')
data_config = yaml.load(config_file, Loader=yaml.FullLoader)
config = data_config[args.data_name]

args.root_path = os.path.join(args.root_path, config["filename"])
args.sampling_freq = config["sampling_freq"]
args.num_classes = config["num_classes"]
window_seconds = config["window_seconds"]
args.windowsize = int(window_seconds * args.sampling_freq)
args.input_length = args.windowsize
# input information
args.c_in = config["num_channels"]
args.device = 'bluesense-RWR1'

if args.difference:
    args.c_in = args.c_in * 2

if args.wavelet_filtering:

    if args.windowsize % 2 == 1:
        N_ds = int(torch.log2(torch.tensor(args.windowsize - 1)).floor()) - 2
    else:
        N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

    args.f_in = args.number_wavelet_filtering * N_ds + 1
else:
    args.f_in = 1


In [4]:
args.model_type = "tinyhar"  #"deepconvlstm"#"sahar" #"deepconvlstm"

args.cross_channel_interaction_type = "attn"
args.cross_channel_aggregation_type = "FC"
args.temporal_info_interaction_type = "lstm"
args.temporal_info_aggregation_type = "tnaive"
exp = Exp(args)
macs, params = get_model_complexity_info(exp.model, (1, args.input_length, args.c_in), as_strings=False,
                                         print_per_layer_stat=True, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))
exp.train()

INFO:root:Use GPU: cuda:0


Build the TinyHAR model!


INFO:root:Done!
INFO:root:Parameter :24864
INFO:root:Set the seed as : 1


model_builder(
  24.86 k, 99.992% Params, 1.5 MMac, 100.000% MACs, 
  (model): TinyHAR_Model(
    24.86 k, 99.992% Params, 1.5 MMac, 100.000% MACs, 
    (layers_conv): ModuleList(
      (0): Sequential(
        160, 0.644% Params, 66.96 KMac, 4.451% MACs, 
        (0): Conv2d(120, 0.483% Params, 44.64 KMac, 2.967% MACs, 1, 20, kernel_size=(5, 1), stride=(1, 1))
        (1): ReLU(0, 0.000% Params, 7.44 KMac, 0.495% MACs, inplace=True)
        (2): BatchNorm2d(40, 0.161% Params, 14.88 KMac, 0.989% MACs, 20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        2.06 k, 8.285% Params, 374.4 KMac, 24.887% MACs, 
        (0): Conv2d(2.02 k, 8.124% Params, 363.6 KMac, 24.169% MACs, 20, 20, kernel_size=(5, 1), stride=(2, 1))
        (1): ReLU(0, 0.000% Params, 3.6 KMac, 0.239% MACs, inplace=True)
        (2): BatchNorm2d(40, 0.161% Params, 7.2 KMac, 0.479% MACs, 20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2

INFO:root:================ LOCV Mode ====================
INFO:root:================ 8 CV ======================
INFO:root:================ the 0 th CV Experiment ================ 


...

INFO:root:Epoch: 1 cost time: 39.48067307472229


step time: 0.30110883712768555


INFO:root:Validation for Epoch: 1 cost time: 1.759477138519287
INFO:root:VALI: Epoch: 1, Steps: 72 | Train Loss: 0.3095434  Vali Loss: 0.1682325 Vali Accuracy: 0.9384766  Vali weighted F1: 0.9398876  Vali macro F1 0.9277856 


Validation loss decreased (inf --> 0.168232).  Saving model ...
0
step time: 0.10352587699890137
1
step time: 0.10303831100463867
2
step time: 0.20806455612182617
3
step time: 0.23105669021606445
4
step time: 0.09752154350280762
5
step time: 0.18105506896972656
6
step time: 0.12352871894836426
7
step time: 0.18204021453857422
8
step time: 0.08106327056884766
9
step time: 0.1245276927947998
10
step time: 0.18804192543029785
11
step time: 0.16304636001586914
12
step time: 0.14103913307189941
13
step time: 0.09400558471679688
14
step time: 0.2020421028137207
15
step time: 0.18453121185302734
16
step time: 0.22655558586120605
17
step time: 0.08552694320678711
18
step time: 0.12851810455322266
19
step time: 0.16203594207763672
20
step time: 0.11351728439331055
21
step time: 0.1790330410003662
22
step time: 0.13691473007202148
23
step time: 0.28856658935546875
24
step time: 0.1315147876739502
25
step time: 0.14652514457702637
26
step time: 0.17803096771240234
27
step time: 0.1505384445190429

KeyboardInterrupt: 

In [ ]:
args.model_type = "deepconvlstm"

exp = Exp(args)
macs, params = get_model_complexity_info(exp.model, (1, args.input_length, args.c_in), as_strings=False,
                                         print_per_layer_stat=True, verbose=False)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

exp.train()